In [ ]:
import os
import pathlib
from dotenv import load_dotenv
import boto3
import json
import warnings

import pandas as pd
import numpy as np
import osc_ingest_trino as osc
import trino
from sqlalchemy.engine import create_engine

import ITR

from ITR.configs import ITR_median, ITR_mean
from ITR.data.data_warehouse import DataWarehouse
from ITR.portfolio_aggregation import PortfolioAggregationMethod
from ITR.temperature_score import TemperatureScore

from ITR.data.base_providers import BaseProviderProductionBenchmark, BaseProviderIntensityBenchmark
from ITR.data.template import TemplateProviderCompany
from ITR.data.vault_providers import dequantify_df
from ITR.interfaces import EScope, ETimeFrames, EScoreResultType, IEIBenchmarkScopes, IProductionBenchmarkScopes, ProjectionControls
# from ITR.configs import LoggingConfig

from ITR.data.osc_units import ureg, Q_, PA_, asPintSeries
from pint import Quantity
from pint_pandas import PintArray, PintType

import logging

In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s') # LoggingConfig.FORMAT
stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

logger.info("Start!")

examples_dir ='' #'examples'
data_dir="data"
data_json_units_dir="json-units"
root = os.path.abspath('')

company_data_path = os.path.join(root, examples_dir, data_dir, "20230106 ITR V2 Sample Data.xlsx")

# Production benchmark (there's only one, and we have to stretch it from OECM to cover TPI)
benchmark_prod_json_file = "benchmark_production_OECM.json"
benchmark_prod_json = os.path.join(root, examples_dir, data_dir, data_json_units_dir, benchmark_prod_json_file)
with open(benchmark_prod_json) as json_file:
    parsed_json = json.load(json_file)

oil_prod_bm = [bm for bm in parsed_json['AnyScope']['benchmarks'] if bm['sector']=='Oil'][0]
oil_base_prod = ureg(oil_prod_bm['base_year_production']).to('MJ')
oil_prod_series = pd.DataFrame({**{'year': [oil_prod['year'] for oil_prod in oil_prod_bm['projections_nounits']]},
                                **{'value': [oil_prod['value'] for oil_prod in oil_prod_bm['projections_nounits']]}}).set_index('year').squeeze()
oil_prod_series = oil_prod_series.add(1).cumprod().mul(oil_base_prod.m)

gas_prod_bm = [bm for bm in parsed_json['AnyScope']['benchmarks'] if bm['sector']=='Gas'][0]
gas_base_prod = ureg(gas_prod_bm['base_year_production']).to('MJ')
gas_prod_series = pd.DataFrame({**{'year': [gas_prod['year'] for gas_prod in gas_prod_bm['projections_nounits']]},
                                **{'value': [gas_prod['value'] for gas_prod in gas_prod_bm['projections_nounits']]}}).set_index('year').squeeze()
gas_prod_series = gas_prod_series.add(1).cumprod().mul(gas_base_prod.m)

oil_and_gas_prod_series = oil_prod_series.add(gas_prod_series).div(oil_base_prod.m + gas_base_prod.m)
oil_and_gas_prod_series = oil_and_gas_prod_series.div(oil_and_gas_prod_series.shift(1))
oil_and_gas_prod_series.iloc[0] = 1.0
oil_and_gas_prod_series = oil_and_gas_prod_series.sub(1)

oil_and_gas_bm = dict(oil_prod_bm)
oil_and_gas_bm['sector'] = 'Oil & Gas'
oil_and_gas_bm['base_year_production'] = f"{oil_base_prod + gas_base_prod:~P}"
oil_and_gas_bm['projections_nounits'] = [ {'year': year, 'value': value} for year,value in oil_and_gas_prod_series.to_dict().items() ]
parsed_json['AnyScope']['benchmarks'].append(oil_and_gas_bm)

# coal_prod_bm = dict([bm for bm in parsed_json['AnyScope']['benchmarks'] if bm['sector']=='Coal'][0])
# coal_base_prod = ureg(coal_prod_bm['base_year_production']).to('MJ')

# coal_prod_bm['sector'] = 'Diversified Mining'
# coal_prod_bm['base_year_production'] = f"{coal_base_prod:~P}"
# parsed_json['AnyScope']['benchmarks'].append(coal_prod_bm)

prod_bms = IProductionBenchmarkScopes.parse_obj(parsed_json)
base_production_bm = BaseProviderProductionBenchmark(production_benchmarks=prod_bms)
logger.info('Load production benchmark from {}'.format(benchmark_prod_json_file))

# Emission intensities
benchmark_EI_OECM_PC_file = "benchmark_EI_OECM_PC.json"
benchmark_EI_OECM_S3_file = "benchmark_EI_OECM_S3.json"
benchmark_EI_OECM_file = "benchmark_EI_OECM.json" # Deprecated!
benchmark_EI_TPI_15_file = "benchmark_EI_TPI_1_5_degrees.json"
benchmark_EI_TPI_file = "benchmark_EI_TPI_2_degrees.json"
benchmark_EI_TPI_below_2_file = "benchmark_EI_TPI_below_2_degrees.json"
benchmark_EI_TPI_2deg_high_efficiency_file = "benchmark_EI_TPI_2_degrees_high_efficiency.json"
benchmark_EI_TPI_2deg_shift_improve_file = "benchmark_EI_TPI_2_degrees_shift_improve.json"

# loading dummy portfolio
df_portfolio = pd.read_excel(company_data_path, sheet_name="Portfolio")
companies = ITR.utils.dataframe_to_portfolio(df_portfolio)
logger.info('Load dummy portfolio from {}. You could upload your own portfolio using the template.'.format(company_data_path))

In [ ]:
eibm = 'OECM_S3'

if eibm == 'OECM_PC':
    benchmark_file = benchmark_EI_OECM_PC_file
elif eibm == 'OECM_S3':
    benchmark_file = benchmark_EI_OECM_S3_file
elif eibm.startswith('TPI_2_degrees'):
    benchmark_file = benchmark_EI_TPI_file
elif eibm == 'TPI_15_degrees':
    benchmark_file = benchmark_EI_TPI_15_file
elif eibm == 'OECM':
    benchmark_file = benchmark_EI_OECM_file
    logger.info('OECM benchmark is for backward compatibility only.  Use OECM_PC instead.')
else:
    benchmark_file = benchmark_EI_TPI_below_2_file
# load intensity benchmarks
benchmark_EI = os.path.join(root, examples_dir, data_dir, data_json_units_dir, benchmark_file)
with open(benchmark_EI) as json_file:
    parsed_json = json.load(json_file)
if eibm.startswith('TPI_2_degrees'):
    extra_EI = os.path.join(root, examples_dir, data_dir, data_json_units_dir,
                            benchmark_EI_TPI_2deg_high_efficiency_file if '_high_efficiency' in eibm
                            else benchmark_EI_TPI_2deg_shift_improve_file)
    with open(extra_EI) as json_file:
        extra_json = json.load(json_file)
        for scope_name in EScope.get_scopes():
            if scope_name in extra_json:
                if scope_name not in parsed_json:
                    parsed_json[scope_name] = extra_json[scope_name]
                else:
                    parsed_json[scope_name]['benchmarks'] += extra_json[scope_name]['benchmarks']
EI_bm = BaseProviderIntensityBenchmark(EI_benchmarks=IEIBenchmarkScopes.parse_obj(parsed_json))

In [ ]:
template_company_data = TemplateProviderCompany(company_data_path, projection_controls = ProjectionControls())
Warehouse = DataWarehouse(template_company_data, benchmark_projected_production=None, benchmarks_projected_ei=None,
                          estimate_missing_data=DataWarehouse.estimate_missing_s3_data)

In [ ]:
# This updates benchmarks and all that depends on them (including trajectories)
Warehouse.update_benchmarks(base_production_bm, EI_bm)

In [ ]:
# Load some standard environment variables from a dot-env file, if it exists.
# If no such file can be found, does not fail, and so allows these environment vars to
# be populated in some other way
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [ ]:
print("Initializing Dev tables")

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ['TRINO_USER_USER1'],
    host = os.environ['TRINO_HOST'],
    port = os.environ['TRINO_PORT']
)

ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'demo_dv'
itr_prefix = 'template_'

sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD_USER1']),
    'http_scheme': 'https',
    'catalog': ingest_catalog,
    'schema': ingest_schema,
}

dev_engine = create_engine(sqlstring, connect_args = sqlargs)
print("connecting with engine " + str(dev_engine))
qres = osc._do_sql(f"show tables in {ingest_schema}", dev_engine, verbose=True)

In [ ]:
# bucket must be configured with credentials for trino, and accessible to the hive catalog
# You may need to use a different prefix here depending on how you name your credentials.env variables
hive_bucket = osc.attach_s3_bucket('S3_HIVE')

hive_catalog = 'osc_datacommons_hive_ingest'
hive_schema = 'ingest'

# This will have identity of main notebook user, not OS-Climate-User1
ingest_engine = osc.attach_trino_engine(verbose=True, catalog=ingest_catalog, schema=ingest_schema)

## ITR Company Data

In [ ]:
df = template_company_data.df_fundamentals[[
    'company_name', 'company_lei', 'company_id',
    'sector', 'country', 'region', 'exposure', 'currency',
    'report_date',
    'company_market_cap', 'company_revenue',
    'company_enterprise_value', 'company_ev_plus_cash',
    'company_total_assets',
    'cash',
    'debt' ]].convert_dtypes()

df.cash = df.cash.astype("Float64")
df.debt = df.debt.astype("Float64")
df.rename(columns={'company_enterprise_value':'company_ev', 'company_ev_plus_cash':'company_evic', 'cash':'company_cash_equivalents', 'debt':'company_debt'}, inplace=True)
df['year'] = df.report_date.dt.year
df.drop(columns='report_date', inplace=True)

schema = osc.create_table_schema_pairs(df)

company_tablename = f"{itr_prefix}company_data"

osc._do_sql(f"drop table if exists {ingest_catalog}.{ingest_schema}.{company_tablename}", ingest_engine, verbose=True)


tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{company_tablename}(
{schema}
) with (
    format = 'ORC',
    partitioning = ARRAY['year']
)
"""

osc._do_sql(tabledef, ingest_engine, verbose=True)

df.to_sql(company_tablename, ingest_engine, schema=ingest_schema, if_exists="append",
          index=False,
          method=osc.TrinoBatchInsert(batch_size = 1000, verbose = True))

In [ ]:
company_info_at_base_year = template_company_data.get_company_intensity_and_production_at_base_year([company_id for company_id in template_company_data.df_fundamentals.company_id.values if company_id !='US6745991058-chem'])

import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    # See https://github.com/hgrecco/pint-pandas/issues/128
    projected_production = Warehouse.benchmark_projected_production.get_company_projected_production(
        company_info_at_base_year)

In [ ]:
production_tablename = f"{itr_prefix}production_data"

osc._do_sql(f"drop table if exists {ingest_catalog}.{ingest_schema}.{production_tablename}", ingest_engine, verbose=True)


production_tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{production_tablename}(
    company_name varchar,
    company_lei varchar,
    company_id varchar,
    sector varchar,
    year bigint,
    production_by_year double,
    production_by_year_units varchar) with (
    format = 'ORC',
    partitioning = ARRAY['year']
)
"""

osc._do_sql(production_tabledef, ingest_engine, verbose=True)

# schema = osc.create_table_schema_pairs(production_data)

rename_year_columns={ y: f"y{y}" for y in range(2000,2100)}
df = projected_production.loc[(slice(None), EScope.S1), :].droplevel('scope')
df = df.rename(columns=rename_year_columns).reset_index()
df2 = pd.wide_to_long(df, stubnames='y', i='company_id', j='year')
df2 = df2.rename(columns={'y':'production_by_year'}).reset_index('year')
df3 = df2.merge(template_company_data.df_fundamentals[['company_name', 'company_lei', 'sector']], on='company_id')
production_df = df3
df4 = dequantify_df(df3).reset_index()
df4 = df4[['company_name', 'company_lei', 'company_id', 'sector', 'year', 'production_by_year', 'production_by_year_units']]

df4.to_sql(production_tablename, ingest_engine, schema=ingest_schema, if_exists="append",
           index=False,
           method=osc.TrinoBatchInsert(batch_size = 4000, verbose = True))

In [ ]:
target_tablename = f"{itr_prefix}target_data"
trajectory_tablename = f"{itr_prefix}trajectory_data"

osc._do_sql(f"drop table if exists {ingest_catalog}.{ingest_schema}.{target_tablename}", ingest_engine, verbose=True)
osc._do_sql(f"drop table if exists {ingest_catalog}.{ingest_schema}.{trajectory_tablename}", ingest_engine, verbose=True)

# schema = osc.create_table_schema_pairs(target_data)

ei_schema = ',\n'.join([f"ei_{scope.lower()}_by_year double, ei_{scope.lower()}_by_year_units varchar" for scope in EScope.get_scopes()])

target_tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{target_tablename}(
    company_name varchar,
    company_lei varchar,
    company_id varchar,
    sector varchar,
    year bigint,
    {ei_schema}) with (
    format = 'ORC',
    partitioning = ARRAY['year']
)
"""
trajectory_tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{trajectory_tablename}(
    company_name varchar,
    company_lei varchar,
    company_id varchar,
    sector varchar,
    year bigint,
    {ei_schema}) with (
    format = 'ORC',
    partitioning = ARRAY['year']
)
"""

osc._do_sql(target_tabledef, ingest_engine, verbose=True)
osc._do_sql(trajectory_tabledef, ingest_engine, verbose=True)

In [ ]:
target_dfs = []
trajectory_dfs = []
projection_tablename = [ target_tablename, trajectory_tablename ]

for i, projection in enumerate(['projected_targets', 'projected_intensities']):
    projection_dfs = []
    for company in template_company_data._companies:
        ei_dict = {}
        for scope in EScope.get_scopes():
            if getattr(company, projection)[scope]:
                ei_dict[scope] = getattr(company, projection)[scope].projections
            else:
                ei_dict[scope] = pd.Series(dtype='object')
        ei_data = pd.concat([ei_dict[scope] for scope in EScope.get_scopes()], axis=1).reset_index()
        ei_data.columns = ['year'] + [f"ei_{scope.lower()}_by_year" for scope in EScope.get_scopes()]
        df = pd.DataFrame(data=[[company.company_name, '', company.company_id, company.sector] for i in ei_data.index],
                          columns=['company_name', 'company_lei', 'company_id', 'sector']).convert_dtypes()
        projection_dfs.append(pd.concat([df, ei_data], axis=1))
    df2 = pd.concat(projection_dfs).reset_index(drop=True).convert_dtypes()
    if projection_tablename[i]==target_tablename:
        target_df = df2
    df3 = dequantify_df(df2)
    df3.to_sql(projection_tablename[i], ingest_engine, schema=ingest_schema,
               if_exists="append", index=False,
               method=osc.TrinoBatchInsert(batch_size = 4000, verbose = True))

osc._do_sql(f"select count (*) from {target_tablename}", ingest_engine, verbose=True)
osc._do_sql(f"select count (*) from {trajectory_tablename}", ingest_engine, verbose=True)

In [ ]:
emissions_tablename = f"{itr_prefix}emissions_data"

osc._do_sql(f"drop table if exists {ingest_catalog}.{ingest_schema}.{emissions_tablename}", ingest_engine, verbose=True)

# schema = osc.create_table_schema_pairs(target_data)

co2_schema = ',\n'.join([f"co2_{scope.lower()}_by_year double, co2_{scope.lower()}_by_year_units varchar" for scope in EScope.get_scopes()])

emissions_tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{emissions_tablename}(
    company_name varchar,
    company_lei varchar,
    company_id varchar,
    sector varchar,
    year bigint,
    {co2_schema}) with (
    format = 'ORC',
    partitioning = ARRAY['year']
)
"""

# Create emissions_data table using production_df and math
emissions_df = production_df.merge(target_df.drop(columns=['company_name', 'company_lei', 'sector']), on=['company_id', 'year'])
emissions_df = emissions_df[~emissions_df.company_id.isin(['DE000SYM9999', 'NO0010657505', 'GB0000961622', 'DE000BASF111', 'IE00BZ12WP82', 'FR0004024222'])]
print(emissions_df.index.names)
for scope in EScope.get_scopes():
    mask = emissions_df[f"ei_{scope.lower()}_by_year"].isna()
    emissions_df.loc[mask, f"ei_{scope.lower()}_by_year"] = emissions_df['production_by_year'].map(lambda x: Q_(np.nan, f"Mt CO2 / ({str(x.u)})"))
    emissions_df[f"co2_{scope.lower()}_by_year"] = emissions_df['production_by_year'].mul(emissions_df[f"ei_{scope.lower()}_by_year"]).astype("pint[Mt CO2e]")
    emissions_df = emissions_df.drop(columns=f"ei_{scope.lower()}_by_year")
emissions_df = emissions_df.drop(columns='production_by_year')
df = dequantify_df(emissions_df)
df.to_sql(emissions_tablename, ingest_engine, schema=ingest_schema,
          if_exists="append", index=False,
          method=osc.TrinoBatchInsert(batch_size = 4000, verbose = True))

In [ ]:
emissions_df